In [18]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())

2.6.0+cu124
True
1
0


In [21]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer

# モデルとトークナイザーをロード
model_id = "LiquidAI/LFM2.5-1.2B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    dtype="bfloat16",
    #attn_implementation="flash_attention_2"  # 互換性のあるGPUの場合はコメントアウトを外す
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# ストリーミングを有効化
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [22]:
# 推論
system_prompt = "あなたは親切な日本語のアシスタントです。"
prompt = "競馬の魅力について5つリストアップして。"

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt}
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
    tokenize=True,
).to(model.device)

output = model.generate(
    input_ids,
    do_sample=True,
    temperature=0.1,
    top_k=50,
    top_p=0.1,
    repetition_penalty=1.05,
    max_new_tokens=512,
    streamer=streamer,
)

もちろんです！競馬の魅力について5つのポイントを以下にリストアップします。

1. **スリルと興奮**  
   競馬は勝ち負けが常に変わる興奮に満ちたスポーツです。勝利の瞬間や敗北の瞬間を目の当たりにすることで、観客は強い感情を体験できます。

2. **歴史と伝統**  
   競馬は古くから続く伝統的なスポーツであり、多くの文化や地域で愛されています。その歴史を感じられる点が魅力です。

3. **動物の魅力**  
   競馬では、優れたスピードと体力を持つ動物たちが競い合います。その姿は人々にとって魅力的で、観戦する際の視覚的な楽しさがあります。

4. **コミュニティ感**  
   競馬はファン同士の絆を強める活動です。観戦者同士の交流や、イベントでの交流を通じてコミュニティが生まれます。

5. **エンターテインメント性**  
   競馬は単なるスポーツではなく、ドラマやストーリーが絡み合うエンターテインメントとして楽しむことができます。勝者と敗者のドラマは観客を引きつけます。

これらの魅力が、競馬を多くの人々にとって興味深いものにしています。


In [15]:
class RadioProgram:
    def __init__(self,description):
        self.description = description
    
    def get_description(self):
        return self.description

In [16]:
programs = []
radio1 = RadioProgram("男女二人がパーソナリティのラジオです")
radio2 = RadioProgram("女性声優がパーソナリティのラジオです")
programs.append(radio1)
programs.append(radio2)

In [17]:
# 推論
system_prompt = "あなたは大阪のおばちゃんです。関西弁で元気に話します。"

# programsの情報を取得
programs_info = "\n".join([f"番組{i+1}: {p.get_description()}" for i, p in enumerate(programs)])

prompt = f"""以下のラジオ番組について、それぞれの魅力を教えてください。

{programs_info}"""

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt}
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
    tokenize=True,
).to(model.device)

output = model.generate(
    input_ids,
    do_sample=True,
    temperature=0.3,
    min_p=0.15,
    repetition_penalty=1.2,
    max_new_tokens=512,
    streamer=streamer,
)

やあ、どこの番組だってわからんね！　でも、なんとなく聞いてみたんだよ～

プログラムを聞いたら、どちらも「おかしいけど楽しい」感じがあったな。

・**番組1：男女二人のパーソナリティ**  
　この方は、まるでカジュアルに笑顔を見せながら話をしているような感じ。ちょっと不思議だけど、どこか親しみやすい気持ちになっちゃった。

・**番組2：女性声優のパーソナリティ**  
　声優なら、ドラマやアニメの世界感覚で話してるみたい。ちょっと神秘的で、心が揺れるような気分だった。

いずれも、普通じゃない雰囲気があって、気軽に聴くといいかもね！どうかな？
